In [22]:
from pathlib import Path
def gen_file_row(f):
  return {'parent':str(f.parent),'type':f.suffix,'size':f.stat().st_size}

def process_dir(d):
  paths = {}
  dir_size = 0
  try:
    for p in d.iterdir():
      try:
        if p.is_dir():
          res = process_dir(p)
          paths.update(res)
          dir_size += res[str(p)]['size']
        elif p.is_file():
          res = gen_file_row(p)
          paths[str(p)] = res
          dir_size += res['size']
      except (FileNotFoundError, PermissionError):
        continue
  except (FileNotFoundError, PermissionError):
    return paths
  paths[str(d)]={'parent':str(d.parent),'type':'directory','size':dir_size}
  return paths

In [38]:
import pandas as pd
data=process_dir(Path('/tensorflow-1.15.2'))
df = pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index':'path'})
df.loc[df['type'] != 'directory'].nlargest(10,'size')

,path,parent,type,size
0,/tensorflow-1.15.2/python3.7/tensorflow_core/p...,/tensorflow-1.15.2/python3.7/tensorflow_core/p...,.so,1001409816
1,/tensorflow-1.15.2/python3.7/tensorflow_core/l...,/tensorflow-1.15.2/python3.7/tensorflow_core,.1,36157560
2,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,17834576
3,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,16647056
4,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,10332280
5,/tensorflow-1.15.2/python3.7/mpi4py/MPI.cpytho...,/tensorflow-1.15.2/python3.7/mpi4py,.so,7001256
6,/tensorflow-1.15.2/python3.7/pretty_midi/TimGM...,/tensorflow-1.15.2/python3.7/pretty_midi,.sf2,5994284
7,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,4735504
8,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,4503280
9,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,/tensorflow-1.15.2/python3.7/tensorflow_core/c...,.so,4486640


In [45]:
import plotly.express as px
fig = px.treemap(df,
    branchvalues = "total",
    names = 'path',
    parents = 'parent',
    values = 'size'
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()